# Bloom Filters

Bloom filter is a data structure that can check if an element is **not** in the list in O(1) and on average much faster than an unsuccessful check of an element in open-addressing or chained hash tables. 

In [4]:
from random import randint

class BloomFilter:
# Private instances
    __hash_init_vects = []
    
    

    def __generate_hash_f(self):
        for i in range(self.__num_filt):
            __hash_init_vects.append(randint(1, 100))
    
# Public instances
    def __init__(self, arr_sz, num_filt, num_elem):
        self.__num_elem = arr_sz / 2 # check
        self.__storage_arr = [] * arr_sz
        if num_filt != None:
            self.__num_filt = num_filt
        else:
            self.__num_filt = arr_sz / num_elem * log(2)
        self.__generate_hash_f()
        
    def add_elem(self, elem):
        
    def search_elem(self, elem):
        if __storage_arr

